In [2]:
import pandas as pd

In [7]:
data=pd.read_csv('D:\pythonProject\pyspark\data\passed_weather_ALL.csv')
data

,rain1h,rain24h,rain12h,rain6h,temperature,tempDiff,humidity,pressure,windDirection,windSpeed,time,city_code,province,city_name,city_index
0,0.0,9999.0,9999.0,9999.0,6.0,NaN,34.0,1021.0,118.0,1.6,2023-12-09 17:00,54511,北京市,北京,1
1,0.0,9999.0,9999.0,9999.0,7.1,NaN,32.0,1020.0,166.0,2.5,2023-12-09 16:00,54511,北京市,北京,1
2,0.0,9999.0,9999.0,9999.0,7.9,NaN,31.0,1020.0,121.0,3.2,2023-12-09 15:00,54511,北京市,北京,1
3,0.0,9999.0,9999.0,9999.0,8.1,NaN,33.0,1019.0,82.0,1.6,2023-12-09 14:00,54511,北京市,北京,1
4,0.0,9999.0,9999.0,9999.0,6.7,NaN,38.0,1019.0,101.0,2.4,2023-12-09 13:00,54511,北京市,北京,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56395,0.0,9999.0,9999.0,9999.0,1.6,NaN,46.0,884.0,317.0,0.8,2023-12-08 22:00,51716,新疆维吾尔自治区,巴楚,2350
56396,0.0,9999.0,9999.0,9999.0,2.3,NaN,46.0,883.0,357.0,1.4,2023-12-08 21:00,51716,新疆维吾尔自治区,巴楚,2350
56397,0.0,9999.0,9999.0,9999.0,3.4,NaN,42.0,883.0,49.0,2.7,2023-12-08 20:00,51716,新疆维吾尔自治区,巴楚,2350
56398,0.0,9999.0,9999.0,9999.0,4.7,NaN,38.0,882.0,39.0,1.5,2023-12-08 19:00,51716,新疆维吾尔自治区,巴楚,2350


计算各城市过去24小时累积雨量

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, desc
from pyspark.sql.types import DecimalType

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .config("spark.executorEnv.PYTHONHASHSEED", "0") \
    .config("spark.hadoop.fs.file.impl", "org.apache.hadoop.fs.RawLocalFileSystem") \
    .getOrCreate()
def passed_rain_analyse(filename):

   print("开始分析累积降雨量")
   spark = SparkSession.builder.config("spark.hadoop.fs.file.impl",
                                       "org.apache.hadoop.fs.RawLocalFileSystem").getOrCreate()

   # spark = SparkSession.builder.getOrCreate()

   df = spark.read.csv(filename, header=True)

   df_rain = df.select(col('province'), col('city_name'), col('city_code'),
                    col('rain1h').cast(DecimalType(scale=1))).filter(col('rain1h') < 1000)

   df_rain_sum = df_rain.groupBy("province", "city_name", "city_code") \
      .agg(sum("rain1h").alias("rain24h")) \
      .sort(desc("rain24h"))

   # df_rain_sum.coalesce(1).write.mode("overwrite").csv("passed_rain_analyse.csv")

   df_rain_sum.coalesce(1).write.csv("passed_rain_analyse.csv")

   print("累积降雨量分析完毕！")

   return df_rain_sum.head(20)  # 前20个

passed_rain_analyse("passed_weather_ALL.csv")

计算各城市过去24小时平均气温

In [5]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, desc
from pyspark.sql.types import DecimalType
from pyspark.sql import functions as F
from pyspark import SparkConf

# 其他代码...


# 其他代码...


def passed_temperature_analyse(filename):

    print("开始分析气温")

    # spark = SparkSession.builder.config(conf=SparkConf()).getOrCreate()
    spark_conf = SparkConf().setAppName("TemperatureAnalysis").setMaster("local[*]")
    spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()


    df = spark.read.csv(filename, header=True)

    df_temperature = df.select(  # 选择需要的列

        df['province'],

        df['city_name'],

        df['city_code'],

        df['temperature'].cast(DecimalType(scale=1)),  # 转换为十进制类型，并将小数点后的位数保留1位

        F.date_format(df['time'], "yyyy-MM-dd").alias("date"),  # 得到日期数据

        F.hour(df['time']).alias("hour")  # 得到小时数据，命名为hour

    ).filter(df['temperature'] < 1000)

    # 筛选四点时次

    df_4point_temperature = df_temperature.filter(df_temperature['hour'].isin([2, 8, 14, 20]))

    df_avg_temperature = df_4point_temperature.groupBy("province", "city_name", "city_code", "date").agg(F.count("temperature"), F.avg("temperature").alias("avg_temperature")).sort(F.asc("avg_temperature")).select("province", "city_name", "city_code", "date",F.format_number('avg_temperature', 1).alias("avg_temperature"))

    df_avg_temperature.show()

    avg_temperature_list = df_avg_temperature.collect()

    # df_avg_temperature.coalesce(1).write.csv("passed_temperature.csv")
    output_path = "D:/pythonProject/pyspark/code/passed_temperature.csv"

    df_avg_temperature.coalesce(1).write.csv(output_path)

    print("气温分析完毕")

    return avg_temperature_list[0:20]  # 前20个
passed_temperature_analyse("passed_weather_ALL.csv")

开始分析气温
+------------+------------+---------+----------+---------------+
|    province|   city_name|city_code|      date|avg_temperature|
+------------+------------+---------+----------+---------------+
|内蒙古自治区|        根河|    50431|2023-12-09|          -32.2|
|    黑龙江省|        新林|    50349|2023-12-09|          -31.6|
|    黑龙江省|        呼中|    50247|2023-12-09|          -30.6|
|    黑龙江省|        漠河|    50136|2023-12-08|          -30.4|
|内蒙古自治区|      图里河|    50434|2023-12-09|          -29.0|
|内蒙古自治区|    额尔古纳|    50425|2023-12-09|          -28.7|
|内蒙古自治区|新巴尔虎左旗|    50618|2023-12-09|          -27.8|
|内蒙古自治区|      满洲里|    50514|2023-12-09|          -27.6|
|内蒙古自治区|      阿尔山|    50727|2023-12-09|          -27.4|
|内蒙古自治区|  陈巴尔虎旗|    50524|2023-12-09|          -27.4|
|    黑龙江省|        漠河|    50136|2023-12-09|          -27.3|
|内蒙古自治区|      牙克石|    50526|2023-12-09|          -27.1|
|内蒙古自治区|      海拉尔|    50527|2023-12-09|          -27.1|
|内蒙古自治区|    鄂温克旗|    50525|2023-12-09|          -26.7|
|内蒙古自治区|

[Row(province='内蒙古自治区', city_name='根河', city_code='50431', date='2023-12-09', avg_temperature='-32.2'),
 Row(province='黑龙江省', city_name='新林', city_code='50349', date='2023-12-09', avg_temperature='-31.6'),
 Row(province='黑龙江省', city_name='呼中', city_code='50247', date='2023-12-09', avg_temperature='-30.6'),
 Row(province='黑龙江省', city_name='漠河', city_code='50136', date='2023-12-08', avg_temperature='-30.4'),
 Row(province='内蒙古自治区', city_name='图里河', city_code='50434', date='2023-12-09', avg_temperature='-29.0'),
 Row(province='内蒙古自治区', city_name='额尔古纳', city_code='50425', date='2023-12-09', avg_temperature='-28.7'),
 Row(province='内蒙古自治区', city_name='新巴尔虎左旗', city_code='50618', date='2023-12-09', avg_temperature='-27.8'),
 Row(province='内蒙古自治区', city_name='满洲里', city_code='50514', date='2023-12-09', avg_temperature='-27.6'),
 Row(province='内蒙古自治区', city_name='阿尔山', city_code='50727', date='2023-12-09', avg_temperature='-27.4'),
 Row(province='内蒙古自治区', city_name='陈巴尔虎旗', city_code='50524',

计算各城市过去24小时平均气压

In [6]:
def passed_pressure_analyse(filename):

    print("开始分析气压")

    spark = SparkSession.builder.config(conf=SparkConf()).getOrCreate()

    df = spark.read.csv(filename, header=True)

    df_pressure = df.select(  # 选择需要的列

        df['province'],

        df['city_name'],

        df['city_code'],

        df['pressure'].cast(DecimalType(scale=1)),

        F.date_format(df['time'], "yyyy-MM-dd").alias("date"),  # 得到日期数据

        F.hour(df['time']).alias("hour")  # 得到小时数据

    )

    df_4point_pressure = df_pressure.filter(df_pressure['hour'].isin([2, 8, 14, 20]))

    df_avg_pressure = df_4point_pressure.groupBy("province", "city_name", "city_code", "date").agg(

        F.count("pressure"), F.avg("pressure").alias("avg_pressure")).sort(

        F.asc("avg_pressure")).select("province", "city_name", "city_code", "date",

                                      F.format_number('avg_pressure', 1).alias("avg_pressure"))

    avg_pressure_list = df_avg_pressure.collect()

    df_avg_pressure.coalesce(1).write.csv("passed_pressure.csv")

    print("气压分析完毕")

    return avg_pressure_list[0:20]  # 最低的20个
passed_pressure_analyse('passed_weather_ALL.csv')

开始分析气压
气压分析完毕


[Row(province='西藏自治区', city_name='珠峰大本营', city_code='55656', date='2023-12-09', avg_pressure='551.0'),
 Row(province='西藏自治区', city_name='珠峰大本营', city_code='55656', date='2023-12-08', avg_pressure='552.0'),
 Row(province='西藏自治区', city_name='双湖', city_code='55178', date='2023-12-09', avg_pressure='552.3'),
 Row(province='西藏自治区', city_name='双湖', city_code='55178', date='2023-12-08', avg_pressure='554.0'),
 Row(province='西藏自治区', city_name='班戈', city_code='55279', date='2023-12-09', avg_pressure='569.3'),
 Row(province='西藏自治区', city_name='安多', city_code='55294', date='2023-12-09', avg_pressure='571.0'),
 Row(province='西藏自治区', city_name='班戈', city_code='55279', date='2023-12-08', avg_pressure='572.0'),
 Row(province='西藏自治区', city_name='措勤', city_code='55357', date='2023-12-09', avg_pressure='572.0'),
 Row(province='西藏自治区', city_name='措勤', city_code='55357', date='2023-12-08', avg_pressure='573.0'),
 Row(province='西藏自治区', city_name='安多', city_code='55294', date='2023-12-08', avg_pressure='573